# 🎤 Agent Vocal IA avec RAG Agentique - Démo Complète
## 100% Local | Temps Réel | Google Colab

---

### 📋 Vue d'ensemble du projet

Ce notebook présente un **agent vocal IA intelligent** capable de :
- 🎙️ **Reconnaissance vocale en temps réel** (Whisper)
- 🤖 **Génération de réponses intelligentes** (Ollama + LLM local)
- 📚 **RAG Agentique** (Retrieval-Augmented Generation avec routage multi-domaines)
- 🔊 **Synthèse vocale naturelle** (Piper TTS)
- ⚡ **Architecture streaming** (Pipecat framework)

### 🎯 Domaines supportés
- **Mathématiques** : Équations, algèbre, géométrie
- **Physique** : Mécanique, optique, électricité
- **Anglais** : Grammaire, conjugaison, vocabulaire

### 🏗️ Architecture
```
Microphone → Whisper STT → Router → RAG → Ollama LLM → Piper TTS → Audio Output
```

---

**⚠️ Important** : Assurez-vous d'avoir activé le GPU dans :
`Runtime → Change runtime type → Hardware accelerator → GPU (T4)`

---

**📊 Temps estimé d'exécution** : ~10 minutes pour l'installation complète

## 📦 Étape 1 : Installation des Dépendances

Cette cellule installe tous les packages nécessaires. Durée : ~5 minutes.

In [ ]:
import sys
import subprocess

print("🚀 Installation des dépendances système...")
print("=" * 70)

# Vérifier si on est sur Colab
try:
    import google.colab
    IS_COLAB = True
    print("✅ Environnement détecté : Google Colab")
except:
    IS_COLAB = False
    print("✅ Environnement détecté : Local/autre")

# Installation des dépendances système
if IS_COLAB:
    !apt-get update -qq
    !apt-get install -y -qq portaudio19-dev python3-pyaudio ffmpeg espeak-ng > /dev/null 2>&1
    print("✅ Dépendances système installées")

# Installation des packages Python
print("\n📦 Installation des packages Python...")
print("-" * 70)

packages = [
    "pipecat-ai",
    "torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118",
    "openai-whisper",
    "faster-whisper",
    "langchain",
    "langchain-community",
    "chromadb",
    "sentence-transformers",
    "faiss-cpu",
    "sounddevice",
    "soundfile",
    "gradio",
    "nest-asyncio",
    "llama-cpp-python",
    "pydub"
]

for i, package in enumerate(packages, 1):
    print(f"[{i}/{len(packages)}] Installation de {package.split()[0]}...")
    subprocess.run(f"{sys.executable} -m pip install -q {package}", shell=True, check=True)

print("\n✅ Toutes les dépendances sont installées!")
print("=" * 70)

## 🔍 Étape 2 : Vérification du GPU

Vérifions que le GPU est bien disponible pour accélérer les modèles.

In [ ]:
import torch

print("🔍 Vérification du matériel...")
print("=" * 70)

# Vérifier CUDA
if torch.cuda.is_available():
    print(f"✅ GPU disponible : {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire GPU : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    DEVICE = "cuda"
else:
    print("⚠️  Pas de GPU détecté - utilisation du CPU (plus lent)")
    DEVICE = "cpu"

print(f"\n🎯 Device utilisé : {DEVICE}")
print("=" * 70)

## 📥 Étape 3 : Clonage du Projet

Téléchargement du code source depuis GitHub.

In [ ]:
import os

print("📥 Clonage du projet depuis GitHub...")
print("=" * 70)

# Nettoyer si existe déjà
if os.path.exists("intelligence_lab_agent_vocal"):
    !rm -rf intelligence_lab_agent_vocal
    print("🧹 Ancien dossier supprimé")

# Cloner le repo
!git clone -b pipecat-local-colab https://github.com/Romainmlt123/agent-vocal-IA-Rag-Agentique.git intelligence_lab_agent_vocal

# Se déplacer dans le dossier
os.chdir("intelligence_lab_agent_vocal")

print("\n✅ Projet cloné avec succès!")
print(f"📁 Dossier actuel : {os.getcwd()}")
print("=" * 70)

## 🤖 Étape 4 : Installation d'Ollama et téléchargement du modèle LLM

Ollama permet d'exécuter des LLM localement. Nous utiliserons Llama 3.2 (1B) optimisé pour Colab.

In [ ]:
import subprocess
import time

print("🤖 Installation d'Ollama...")
print("=" * 70)

# Installer Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Démarrer le serveur Ollama en arrière-plan
print("\n🚀 Démarrage du serveur Ollama...")
ollama_process = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(5)  # Attendre le démarrage

print("✅ Serveur Ollama démarré")

# Télécharger le modèle (version légère pour Colab)
print("\n📥 Téléchargement du modèle Qwen2 1.5B...")
print("   (Optimisé pour Colab, ~900MB)")
!ollama pull qwen2:1.5b

print("\n✅ Modèle LLM prêt!")
print("=" * 70)

## 🎙️ Étape 5 : Téléchargement du modèle Whisper (STT)

Whisper permet la reconnaissance vocale en temps réel.

In [ ]:
from faster_whisper import WhisperModel

print("🎙️ Téléchargement du modèle Whisper...")
print("=" * 70)

# Télécharger le modèle (base = bon compromis)
whisper_model = WhisperModel("base", device=DEVICE, compute_type="float16" if DEVICE == "cuda" else "int8")

print("✅ Modèle Whisper prêt!")
print("=" * 70)

## 📚 Étape 6 : Construction des index RAG

Construction des index vectoriels pour les documents de mathématiques, physique et anglais.

In [ ]:
import sys
sys.path.append('/content/intelligence_lab_agent_vocal')

print("📚 Construction des index RAG...")
print("=" * 70)

# Construire les index pour chaque domaine en utilisant le script legacy
import subprocess
result = subprocess.run(
    [sys.executable, "-m", "src.legacy.rag_build"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print(result.stdout)
    print("\n✅ Index RAG construits!")
    print("   - data/maths/index.faiss")
    print("   - data/physique/index.faiss") 
    print("   - data/anglais/index.faiss")
else:
    print("⚠️ Erreur lors de la construction:")
    print(result.stderr)

print("=" * 70)

## 🎨 Étape 7 : Initialisation des composants

Chargement de tous les modules : Router, RAG, LLM, ASR, TTS.

In [ ]:
from src.pipeline.voice_pipeline import create_voice_pipeline
import asyncio

print("🎨 Initialisation du Pipeline Pipecat...")
print("=" * 70)

# Créer le pipeline orchestrateur avec tous les services intégrés
print("? Création du Voice Pipeline Orchestrator...")
pipeline = await create_voice_pipeline(
    stt_model_size="base",
    llm_model="qwen2:1.5b",
    tts_voice="fr_FR-siwis-medium",
    rag_top_k=4,
    enable_metrics=True
)

print("\n✅ Pipeline Pipecat complètement configuré!")
print("? Architecture : Audio → Whisper → RAG → Ollama → Piper → Audio")
print("=" * 70)
print("\n🎯 Composants du pipeline:")
print(f"  • STT: Whisper {pipeline.stt_model_size}")
print(f"  • LLM: {pipeline.llm_model}")
print(f"  • TTS: {pipeline.tts_voice}")
print(f"  • RAG: Top-{pipeline.rag_top_k} retrieval")
print("=" * 70)


## 🎭 Étape 8 : Interface Gradio - Démo Interactive

Interface permettant au jury de tester l'agent vocal en direct.

In [ ]:
import gradio as gr
import numpy as np

def process_audio(audio_input):
    """
    Traite l'audio enregistré et retourne la réponse de l'agent
    
    Args:
        audio_input: tuple (sample_rate, audio_data) depuis Gradio
        
    Returns:
        tuple: (transcription, sujet, réponse_texte, audio_sortie, sources)
    """
    if audio_input is None:
        return "❌ Aucun audio enregistré", "", "", None, ""
    
    try:
        # Extraire les données
        sample_rate, audio_data = audio_input
        
        # Si stereo, convertir en mono
        if len(audio_data.shape) > 1:
            audio_data = audio_data.mean(axis=1)
        
        # Normaliser
        audio_data = audio_data.astype(np.float32)
        if audio_data.max() > 1.0:
            audio_data = audio_data / 32768.0
        
        # Traiter avec l'orchestrator
        result = orchestrator.process_question(audio_data, sample_rate)
        
        # Formater les sources
        sources_text = "\n\n".join([
            f"📄 **{src['title']}** (score: {src['score']:.2f})\n{src['content'][:200]}..."
            for src in result.get('sources', [])
        ])
        
        return (
            result['transcription'],
            result['subject'],
            result['response'],
            (result['sample_rate'], result['audio']),
            sources_text or "Aucune source utilisée"
        )
        
    except Exception as e:
        return f"❌ Erreur: {str(e)}", "", "", None, ""


# Créer l'interface
with gr.Blocks(title="🎤 Agent Vocal IA - RAG Agentique", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎤 Agent Vocal IA avec RAG Agentique
    ### 100% Local | Temps Réel | Multi-Domaines
    
    **Instructions** :
    1. Cliquez sur le microphone 🎤 pour enregistrer votre question
    2. Parlez clairement (français)
    3. Attendez la transcription et la réponse de l'IA
    4. Écoutez la réponse audio générée 🔊
    
    **Domaines supportés** : Mathématiques 🧮 | Physique ⚛️ | Anglais 🇬🇧
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            # Input
            audio_input = gr.Audio(
                sources=["microphone"],
                type="numpy",
                label="🎙️ Enregistrez votre question"
            )
            submit_btn = gr.Button("🚀 Traiter", variant="primary", size="lg")
            
        with gr.Column(scale=1):
            # Outputs
            transcription_out = gr.Textbox(
                label="📝 Transcription",
                lines=2,
                interactive=False
            )
            subject_out = gr.Textbox(
                label="🎯 Domaine détecté",
                interactive=False
            )
            response_out = gr.Textbox(
                label="💡 Réponse de l'IA",
                lines=5,
                interactive=False
            )
            audio_output = gr.Audio(
                label="🔊 Réponse audio",
                type="numpy"
            )
    
    with gr.Accordion("📚 Sources RAG utilisées", open=False):
        sources_out = gr.Markdown()
    
    # Exemples
    gr.Markdown("### 💡 Exemples de questions")
    gr.Examples(
        examples=[
            "Comment résoudre une équation du second degré ?",
            "Qu'est-ce que la loi de Newton ?",
            "Comment conjuguer le verbe to be au présent ?"
        ],
        inputs=None,
        label="Cliquez pour copier une question exemple"
    )
    
    # Event
    submit_btn.click(
        fn=process_audio,
        inputs=[audio_input],
        outputs=[transcription_out, subject_out, response_out, audio_output, sources_out]
    )

print("✅ Interface Gradio créée!")
print("=" * 70)

## 🚀 Étape 9 : Lancement de l'Application

Lancez l'interface Gradio et testez l'agent vocal!

In [ ]:
print("🚀 Lancement de l'application...")
print("=" * 70)
print("\n📱 L'interface va s'ouvrir dans quelques secondes...")
print("💡 Utilisez le microphone pour poser vos questions!")
print("\n" + "=" * 70)

# Lancer l'application
demo.launch(
    share=True,  # Créer un lien public partageable
    debug=True,
    show_error=True
)